In [1]:
import pandas as pd

data = pd.read_csv("news.csv", usecols=['Body', 'Fake'])
data = data.rename(columns={'Body': 'text', 'Fake': 'label'})

data.head()


,text,label
0,Die Sondierungsgespräche zwischen Union und SP...,1
1,Nun ist es auch medizinisch offiziell bestätig...,1
2,"Es waren zähe Verhandlungen, doch die Freien D...",1
3,Wo treibt sie sich immer bis spät in die Nacht...,1
4,Der Parteivorstand drückt nochmal ein Auge zu:...,1


In [12]:
import re
import nltk
import spacy
import pandas as pd

nlp = spacy.load('de_core_news_sm')

def preprocess_text(text):
    text_lower = text.lower()
    tokens = re.findall(r'\b[a-z0-9äöüß]+\b', text_lower)
    tokens = [token for token in tokens if token not in stop_words]
    cleaned_text = ' '.join(tokens)

    
    doc = nlp(cleaned_text)
    adjective_count = sum(1 for token in doc if token.pos_ == 'ADJ')
    
    adverb_count = sum(1 for token in doc if token.pos_ == 'ADV')
    
    entity_count = len(doc.ents)
    return cleaned_text, adjective_count, adverb_count, entity_count


def sample1(data, n_samples=4627):
    df_real = data[data['label'] == 0]
    df_fake = data[data['label'] == 1]
    
    df_real_sampled = df_real.sample(n=n_samples, random_state=42)
    df_fake_sampled = df_fake.sample(n=n_samples, random_state=42)
    
    data_balanced = pd.concat([df_real_sampled, df_fake_sampled])

    data_balanced = data_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
    
    data_balanced[['cleaned_text', 'adjective_count', 'adverb_count', 'entity_count']] = \
        data_balanced['text'].apply(lambda x: pd.Series(preprocess_text(x)))
    
    return data_balanced

data_balanced = sample1(data)
data_balanced.head()


,text,label,cleaned_text,adjective_count,adverb_count,entity_count
0,Neuartige Übertragungsmethode von Neurostimula...,0,neuartige übertragungsmethode neurostimulation...,44,28,5
1,Mark (25) ist verzweifelt. Heute Morgen hatte ...,1,mark 25 verzweifelt heute morgen elefanten uni...,19,40,11
2,Einen ungewöhnlichen Weg geht man neuerdings b...,1,ungewöhnlichen geht neuerdings beim team stron...,22,28,8
3,Weil Behörden und Tierärzte kein Vertrauen meh...,1,behörden tierärzte vertrauen mehr tier zecke b...,19,28,9
4,iPad 2 wird angeblich mit extrem hoher Auflösu...,0,ipad 2 angeblich extrem hoher auflösung vorleg...,29,43,20


In [22]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

def create_feature_df(data, max_features=None):

    vectorizer = CountVectorizer(
        token_pattern=r'[A-Za-zÄÖÜäöüß]+',
        max_features=max_features
    )
    
    X_counts = vectorizer.fit_transform(data['cleaned_text'])
    
    feature_names = vectorizer.get_feature_names_out()
    bow_df = pd.DataFrame(X_counts.toarray(), columns=feature_names)
    
    bow_df['adjective_count'] = data['adjective_count'].values
    bow_df['adverb_count']    = data['adverb_count'].values
    bow_df['entity_count']    = data['entity_count'].values
    
    return bow_df


features_df = create_feature_df(data_balanced, max_features=2000)
features_df.tail()


,a,ab,abend,abgeordnete,abgeordneten,abgeschlossen,abkommen,abschließend,abschluss,absolut,...,überhaupt,übernahme,übernehmen,überraschend,überrascht,überzeugt,übrigen,adjective_count,adverb_count,entity_count
9249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,40,11,5
9250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,30,36,21
9251,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,27,33,18
9252,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,21,16,10
9253,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,23,20,9


In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = features_df
y = data_balanced['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.97


In [30]:
new_message = """Der US-Präsident Donald Trump wird US-Medienberichten zufolge am Donnerstag eine Anordnung zur Auflösung des Bildungsministeriums auf den Weg bringen. Wie mehrere Medien am Mittwoch berichteten, soll der Erlass während einer Zeremonie im Weißen Haus unterzeichnet werden.

Laut dem Nachrichtenportal "Politico" nehmen an der Unterzeichnung am Donnerstag mehrere republikanische Gouverneure teil.

Inzwischen ist das von der Trump-Vertrauten Linda McMahon geführte Ministerium weiter dabei, die vom Präsidenten und seinem Berater Elon Musk vorangetriebenen Personalkürzungen und Mittelstreichungen umzusetzen."""

new_message_fake = """Schockierende Enthüllung: Wissenschaftler behaupten, dass der Klimawandel 
durch die Freisetzung außerirdischer Substanzen verursacht wurde. In Berlin wurde ein geheimer 
Regierungsbund entdeckt, der angeblich das Wetter manipuliert, um globale Ereignisse zu steuern. 
Experten warnen vor baldigen, unerklärlichen Naturkatastrophen!"""


cleaned_text, adj_count, adv_count, ent_count = preprocess_text(new_message_fake)

vectorizer = CountVectorizer(max_features=2000, token_pattern=r'[A-Za-zÄÖÜäöüß]+')
vectorizer.fit(data_balanced['cleaned_text'])

message_counts = vectorizer.transform([cleaned_text])

import pandas as pd
new_message_df = pd.DataFrame(message_counts.toarray(), columns=vectorizer.get_feature_names_out())
new_message_df['adjective_count'] = adj_count
new_message_df['adverb_count'] = adv_count
new_message_df['entity_count'] = ent_count

prediction = clf.predict(new_message_df)

print(f"{prediction}, (1 = fake, 0 = real)")


[1], (1 = fake, 0 = real)
